## Дополнительное задание (курс "Практикум по программированию на языке Python")

### Тема: Web-сервер для Telegram-бота.

#### Преподаватели: Мурат Апишев (mel-lain@yandex.ru) и Роман Ищенко (roman.ischenko@gmail.com)

**Выдана**:   -

**Дедлайн**:   -

**Среда выполнения**: Jupyter Notebook (Python 3.7)

#### Правила:

Результаты выполнения задания:

- архив со скриптами и файлами Dockerfile, Pipfile и Pipfile.lock, который 1-2 команды позволяет развернуть сервер, решающий поставленные в задании задачи
- Jupyter Notebook, где __весь код__ из скриптов дублируется (1 ячейка - 1 скрипт) с комментарием, содержащим информацию о том, из какого файла взят код и что верхнеуровнево этот код делает
- ссылка на чат-бота в Telegram, который автоматически начинает обработку пользовательских запросов после разворачивания сервера

__Максимальное число баллов за задание - 25__.

Работа делится на базовую и дополнительную части, __полное выполнение базовой является обязательным__! Без этого вся работа автоматически оцениваниется 0 баллов. Дополнительную часть можно решать фрагментарно, оценка будет выставляться пропорционально.

Готовое задание отправляется на почту преподавателя.

Задание выполняется самостоятельно. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов. Если вы нашли в Интернете какой-то специфичный код, который собираетесь заимствовать, обязательно укажите это в задании - наверняка вы не единственный, кто найдёт и использует эту информацию.

Удалять фрагменты формулировок заданий запрещается.

#### Постановка задачи:

- В данной работе нужно написать веб-сервер для обслуживания чат-бота в мессенджере Telegram
- Чат-бот предназначен для обработки изображений, передаваемых пользователем
- Сервер должен быть реализован на Flask + waitress
- Проект разворачивается с помощью библиотеки управления виртуальными окружениями Pipenv и технологии контейнеризации Docker

### Базовая часть (12 баллов)

- Требуется реализовать чат-бота для мессенджера Telegram, который возвращает пользователю на любое входное текстовое сообщение это сообщение. Предполагается, что никакие другие данные (картинки, стикеры) бот получать не будет
- Бэкенд проекта должен быть реализован на Flask + waitress (или FastAPI)
- Для прокидывания сервера наружу требуется использовать сервис и утилиту ngrok (как исполняемы файл, не pyngrok)
- Проект должен разворачиваться с помощью Docker, внутри которого создается виртуальное окружение с помощью Pipenv, решение должно разворачиваться командами:

```
docker build -t tg-bot .
docker run -d tg-bot
```

- __Обязательно (!!!)__ нужно вставить копии содержимого всех файлов проекта в этот ноутбук (1 файл - 1 ячейка) и подписать, к какому файлу относится какая ячейка и что код в ней в общих чертах делает

__Рекомендации:__

1. [Ссылка](https://core.telegram.org/bots) на инструкцию по созданию ботов в Telegram

2. Для работы с Telegram лучше всего использовать библиотеку python-telegram-bot

3. Рекомендуемая сигнатура для обработчика POST-запросов от бота:

In [ ]:
# app = Flask(__name__)

@app.route(f'/{BOT_TOKEN}', methods=['POST'])

4. Рекомендуемая структура проекта (плоская):
    - Dockerfile
    - Pipfile
    - Pipfile.lock
    - settings.py (для хранения токена, локального адреса сервера \[localhost + port\] и любых иных общих статических данных)
    - server.py (для старта локального сервера получения/отправки запросов)
    - configure_and_run.py (для настройки и старта всего сервиса)

    Опционально можно завести отдельные файлы для логики обработчика запросов, хранения текстов сообщений и любых иных целей


5. Dockerfile может иметь следующую структуру:

```
FROM python:3.7.9-stretch
# установка нужных утилит в систему, например, curl и unzip
# установка pipenv
WORKDIR /app
# копирование всех важных файлов в рабочую директорию
# запуск pipenv
ENTRYPOINT ["pipenv", "run", "python", "configure_and_run.py", "&"]
```

Последняя команда будет запускаться при вызове `docker run`

6. Вывод информации для отладки и логгирования в stdout можно делать с помощью `os.system(f'echo {message}')`

7. Поскольку локально поднятый сервер не виден снаружи, нужно как-то прокинуть его в Интернет, это можно сделать с помощью облачного хостинга или сервиса ngrok, который можно использовать через одноименную консольную утилиту, что и требуется в задании

8. Выкачивание и запуск ngrok можно выполнить в configure_and_run.py, в этом помогут утилиты curl и unzip. Для этих и других задач, требующих параллельного вызова сторонних приложений и скриптов, полезен модуль subprocess

9. После того, как ngrok будет запущен на указанном локальном адресе, нужно будет передать внешний адрес боту (выставить webhook). Достать этот адрес можно, запустив следующую команду и распарсив выходной файл (там будет json):


In [ ]:
os.system(f'curl http://localhost:4040/api/tunnels > {filename}')

10. После этого выставить webhook можно в server.py после создания и перед запуском Flask-сервера командой

In [ ]:
# bot = telegram.Bot(BOT_TOKEN)

bot.setWebhook('{URL}/{HOOK}'.format(URL=EXTERNAL_URL, HOOK=BOT_TOKEN))

11. Обработку запросов стоит оборачивать в try-except-блок и отправлять пользователю валидное сообщение об ошибке или просьбу повторить какое-то действие, саму ошибку при этому надо логгировать

12. Все вспомогательные файлы, возникающие в процессе работы, должны гарантированно удаляться, как только надобность в них отпадает

13. Запуск сервера имеет смысл производить из configure_and_run.py с помощью subprocess (полезен будет метод communicate)

In [ ]:
"""
file: server.py
Реализация телеграм бота при помощи библиотеки pytelegrambotapi, работающий через webhook
"""
import os
import time

import yaml
import requests

from telebot import TeleBot, types
from subprocess import Popen, PIPE
from flask import Flask, request, abort


# Логгирование: время + сообщение
def logger(message):
    cur_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    os.system(f'echo LOGGER:\t{cur_time}\t{message}')


# Запуск сервиса Ngrok через subprocess.Popen
# Возвращает ссылку сгенерированную Ngrok сервисом
def run_ngrok():
    logger("Starting Ngrok service")
    _ = Popen(['ngrok', 'http', str(PORT)], stdout=PIPE)
    logger(f"Waiting 10 seconds")
    time.sleep(10)
    url = None
    try:
        response = requests.get('http://localhost:4040/api/tunnels').json()
        url = response['tunnels'][0]['public_url']
        logger(f"Ngrok url: {url}")
    except Exception as e:
        logger(f"Cant get Ngrok URL with message: {e}")
    return url


# Получаем переменные из конфигурационного файла
with open("config.yml", "r") as f:
    os.system(f'echo ')
    config = yaml.load(f, yaml.Loader)

TOKEN = config.get('bot').get('token')
HOST = config.get('api').get('host')
PORT = config.get('api').get('port')
DEBUG = config.get('debug')


bot = TeleBot(TOKEN)
app = Flask(__name__)


# Вебхук
@app.route(f'/{TOKEN}', methods=['POST'])
def webhook():
    if request.headers.get('content-type') == 'application/json':
        json_string = request.get_data().decode('utf-8')
        update = types.Update.de_json(json_string)
        bot.process_new_updates([update])
        return ''
    else:
        abort(403)


# Рекция на сообщение "\start"
@bot.message_handler(commands=['start'])
def send_welcome(message):
    if DEBUG:
        logger(f"User: {message.chat.id}\tStart using bot")
    bot.send_message(
        message.chat.id, "Привет, я эхо-бот для курса по Python от MIPT!")


# Реакция на все остальные сообщения, "эхо"
@bot.message_handler(content_types=['text'])
def echo_text(message):
    if DEBUG:
        logger(f"User: {message.chat.id}\tMessage: {message.text}")
    bot.send_message(message.chat.id, message.text)

In [ ]:
"""
file: configure_and_run.py
Запуск бота и сервисов
"""
import yaml
from waitress import serve

from server import bot, app
from server import logger, run_ngrok
from server import TOKEN, HOST, PORT

if __name__ == '__main__':
    logger("Starting web-application")
    bot.remove_webhook()
    ngrok_url = run_ngrok()
    logger("Ngrok service started")
    bot.set_webhook(url=f'{ngrok_url}/{TOKEN}')
    logger("Webhook set")
    logger("Web-application ready!")
    serve(app, host=HOST, port=PORT)

In [ ]:
"""
file: config.yml
Файл хранящий статические данные, при включенном debug будут отображаться сообщения внутри бота
"""
bot:
    token: # токен
api:
    host: 0.0.0.0
    port: 8443
debug: False

In [ ]:
"""
file: Dockerfile
Докерфайл
"""
FROM python:3.9.13-buster

RUN mkdir -p /usr/src/app
ENV CONTAINER_HOME=/usr/src/app
ADD . $CONTAINER_HOME
WORKDIR $CONTAINER_HOME

RUN curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | tee /etc/apt/sources.list.d/ngrok.list && apt update && apt install ngrok
RUN pip install pipenv
RUN pipenv install

ENTRYPOINT ["pipenv", "run", "python", "configure_and_run.py", "&"]

### Дополнительная часть

__Задание 1 (2 балла):__ Доработать базовое решение до бота, который предлагает пользователю отправить изображение в формате JPG или PNG (в виде аттача), реагирует адекватно на попытки послать ему что-то иное, и в случае получения валидного изображения возвращает его в чат ответным сообщением

__Рекомендации:__

1. Нужно аккуратно обрабатывать крайние случаи (пришел файл, не являющийся изображением, или имеющий не тот формат)

2. Полезна функция, которая определяет, что сообщение содержит файл или фото:

In [ ]:
# content = telegram.Update...

@staticmethod
def is_file(content):
    return (content.message is not None and
            ((content.message.document is not None and content.message.document.file_id is not None) or
             (content.message.photo is not None and len(content.message.photo) and content.message.photo[0].file_id is not None)))

In [ ]:
"""
file: server.py
Реализация телеграм бота при помощи библиотеки pytelegrambotapi, работающий через webhook
Изменения: убрал эхо для текста, добавил эхо для изображений и сработывает только на изображениях
"""
import os
import time

import yaml
import requests

from telebot import TeleBot, types
from subprocess import Popen, PIPE
from flask import Flask, request, abort


# Логгирование: время + сообщение
def logger(message):
    cur_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    os.system(f'echo LOGGER:\t{cur_time}\t{message}')


# Запуск сервиса Ngrok через subprocess.Popen
# Возвращает ссылку сгенерированную Ngrok сервисом
def run_ngrok():
    logger("Starting Ngrok service")
    _ = Popen(['ngrok', 'http', str(PORT)], stdout=PIPE)
    logger(f"Waiting 10 seconds")
    time.sleep(10)
    url = None
    try:
        response = requests.get('http://localhost:4040/api/tunnels').json()
        url = response['tunnels'][0]['public_url']
        logger(f"Ngrok url: {url}")
    except Exception as e:
        logger(f"Cant get Ngrok URL with message: {e}")
    return url


# Получаем переменные из конфигурационного файла
with open("config.yml", "r") as f:
    os.system(f'echo ')
    config = yaml.load(f, yaml.Loader)

TOKEN = config.get('bot').get('token')
HOST = config.get('api').get('host')
PORT = config.get('api').get('port')
DEBUG = config.get('debug')


bot = TeleBot(TOKEN)
app = Flask(__name__)

# Типы на которые не должно срабатывать
other_types = [
    'text', 'audio', 'sticker', 'video', 'video_note', 'voice', 'location', 'contact', 'new_chat_members',
    'new_chat_title', 'new_chat_photo', 'delete_chat_photo', 'group_chat_created', 'supergroup_chat_created', 
    'channel_chat_created', 'migrate_to_chat_id', 'migrate_from_chat_id', 'pinned_message', 'left_chat_member'
]


# Вебхук
@app.route(f'/{TOKEN}', methods=['POST'])
def webhook():
    if request.headers.get('content-type') == 'application/json':
        json_string = request.get_data().decode('utf-8')
        update = types.Update.de_json(json_string)
        bot.process_new_updates([update])
        return ''
    else:
        abort(403)


# Рекция на сообщение "\start"
@bot.message_handler(commands=['start'])
def send_welcome(message):
    if DEBUG:
        logger(f"User: {message.chat.id}\tStart using bot")
    bot.send_message(
        message.chat.id, 'Привет, я бот, который может обрабатывать только изображения в формате jpg и png')


# Обработчик для других форматов данных
@bot.message_handler(func=lambda message: True, content_types=other_types)
def other_message(message):
    if DEBUG:
            logger(f"User: {message.chat.id}\tОтправил сообщение в не правильном формате")
    bot.send_message(message.chat.id, 'Не верный формат. Нужно отправить изображение в формате jpg и png')


# Обработчик для изображений, с учетом проверок
@bot.message_handler(func=lambda message: True, content_types=['photo'])
def photo_message(message):
    if message is not None and message.photo is not None and len(message.photo) and \
    message.photo[0].file_id is not None:
        if DEBUG:
            logger(f"User: {message.chat.id}\tОтправил изображение")
        bot.send_photo(message.chat.id, message.photo[0].file_id)
    else:
        if DEBUG:
            logger(f"User: {message.chat.id}\tОтправил изображение в не правильном формате")
        bot.send_message(message.chat.id, 'Не верный формат. Нужно отправить изображение в формате jpg и png')


# Обработчик для attach изображений, с учетом проверок
@bot.message_handler(func=lambda message: True, content_types=['document'])
def doc_message(message):
    ext = message.document.mime_type[-3:]
    if message is not None and message.document is not None and message.document.file_id is not None \
    and (ext == 'png' or ext == 'jpg'):
        if DEBUG:
            logger(f"User: {message.chat.id}\tОтправил изображение")
        bot.send_document(message.chat.id, message.document.file_id)
    else:
        if DEBUG:
            logger(f"User: {message.chat.id}\tОтправил изображение в не правильном формате")
        bot.send_message(message.chat.id, 'Не верный формат. Нужно отправить изображение в формате jpg и png')

__Задание 2 (5 баллов):__ Модернизировать решение из предыдущего пункта - при получении изображения от пользователя нужно вернуть не его, а текстовое сообщение с кнопкой, в котором написано, что при нажатии кнопки полученное ранее изображение будет отправлено пользователю обратно. Это и должно происходить при нажатии на кнопку. При этом пользователь может отправить в ответ еще одно изображение, и тогда он снова получит сообщение с кнопкой, но сохранено будет уже новое изображение.

__Рекомендации:__

1. В отличие от предыдущих пунктов, здесь сессия пользователя начинает содержать состояния: пользователь еще не отправил картинку, и пользователь уже сделал это. Это означает, что стоит завести профиль пользователя, который будет идентифицироваться по `chat_id` (который используется при отправке сообщений) и содержать текущие состояние, картинку или любые иные полезные и вспомогательные данные пользователя

2. Хранить нужно не один общий профиль, а по профилю на ользователя, иначе сервис нельзя будет использоваться нескольким пользователям. Обычно профили хранятся в базах, для простоты реализации в задании достаточно ограничиться хранением в `dict` в оперативной памяти, который заводится при старте сервера

3. Нужно следить за корректностью изменения состояния сессии и профиля пользователя, в том числе и при возникновении различных ошибок

4. Полезна функция, которая определяет, что сообщение отправлено нажатием кнопки:

In [ ]:
@staticmethod
def is_button(content):
    return content.callback_query is not None and content.callback_query.data is not None

In [ ]:
"""
file: server.py
Реализация телеграм бота при помощи библиотеки pytelegrambotapi, работающий через webhook
Изменения: убрал эхо для текста, добавил эхо для изображений и сработывает только на изображениях
Изменения: добавил возможность возвращать не все изображения, а только после нажатия, при этом словарь users
хранит состояния
"""
import os
import time

import yaml
import requests

from telebot import TeleBot, types
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
from subprocess import Popen, PIPE
from flask import Flask, request, abort


# Логгирование: время + сообщение
def logger(message):
    cur_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    os.system(f'echo LOGGER:\t{cur_time}\t{message}')


# Запуск сервиса Ngrok через subprocess.Popen
# Возвращает ссылку сгенерированную Ngrok сервисом
def run_ngrok():
    logger("Starting Ngrok service")
    _ = Popen(['ngrok', 'http', str(PORT)], stdout=PIPE)
    logger(f"Waiting 10 seconds")
    time.sleep(10)
    url = None
    try:
        response = requests.get('http://localhost:4040/api/tunnels').json()
        url = response['tunnels'][0]['public_url']
        logger(f"Ngrok url: {url}")
    except Exception as e:
        logger(f"Cant get Ngrok URL with message: {e}")
    return url


# Получаем переменные из конфигурационного файла
with open("config.yml", "r") as f:
    os.system(f'echo ')
    config = yaml.load(f, yaml.Loader)

TOKEN = config.get('bot').get('token')
HOST = config.get('api').get('host')
PORT = config.get('api').get('port')
DEBUG = config.get('debug')


bot = TeleBot(TOKEN)
app = Flask(__name__)

# Типы на которые не должно срабатывать
other_types = [
    'text', 'audio', 'sticker', 'video', 'video_note', 'voice', 'location', 'contact', 'new_chat_members',
    'new_chat_title', 'new_chat_photo', 'delete_chat_photo', 'group_chat_created', 'supergroup_chat_created',
    'channel_chat_created', 'migrate_to_chat_id', 'migrate_from_chat_id', 'pinned_message', 'left_chat_member'
]
# Словарь пользователей и их состояний
users = {}


# Задаем Inline клавишу и callback для нее
def gen_markup():
    markup = InlineKeyboardMarkup()
    markup.add(InlineKeyboardButton("Отправить изображение обратно", callback_data="send_back"))
    return markup


# Вебхук
@app.route(f'/{TOKEN}', methods=['POST'])
def webhook():
    if request.headers.get('content-type') == 'application/json':
        json_string = request.get_data().decode('utf-8')
        update = types.Update.de_json(json_string)
        bot.process_new_updates([update])
        return ''
    else:
        abort(403)


# Рекция на сообщение "\start"
@bot.message_handler(commands=['start'])
def send_welcome(message):
    chat_id = message.chat.id
    if DEBUG:
        logger(f"User: {chat_id}\tStart using bot")
    users[chat_id] = {}
    bot.send_message(chat_id, 'Привет, я бот, который может обрабатывать только изображения в формате jpg и png')


# Обработчик для других форматов данных
@bot.message_handler(func=lambda message: True, content_types=other_types)
def other_message(message):
    chat_id = message.chat.id
    if chat_id in users:
        if DEBUG:
            logger(f"User: {chat_id}\tОтправил сообщение в не правильном формате")
        bot.send_message(chat_id, 'Не верный формат. Нужно отправить изображение в формате jpg и png')
    else:
        bot.send_message(chat_id, 'Сначала вы должны стартовать бот коммандой /start')

        
# Обработчик для изображений, с учетом проверок
@bot.message_handler(func=lambda message: True, content_types=['photo'])
def photo_message(message):
    chat_id = message.chat.id
    if chat_id in users:
        users[chat_id]['img'] = None
        users[chat_id]['type'] = None
        if message is not None and message.photo is not None and len(message.photo) \
        and message.photo[0].file_id is not None:
            if DEBUG:
                logger(f"User: {chat_id}\tОтправил изображение")
            users[chat_id]['img'] = message.photo[0].file_id
            users[chat_id]['type'] = 'photo'
            bot.send_message(chat_id, 'Нажмите кнопку или отправьте новое изображение', reply_markup=gen_markup())
        else:
            if DEBUG:
                logger(f"User: {chat_id}\tОтправил изображение в не правильном формате")
            bot.send_message(chat_id, 'Не верный формат. Нужно отправить изображение в формате jpg и png')
    else:
        bot.send_message(chat_id, 'Сначала вы должны стартовать бот коммандой /start')


# Обработчик для attach изображений, с учетом проверок
@bot.message_handler(func=lambda message: True, content_types=['document'])
def doc_message(message):
    chat_id = message.chat.id
    if chat_id in users:
        ext = message.document.mime_type[-3:]
        users[chat_id]['img'] = None
        users[chat_id]['type'] = None
        if message is not None and message.document is not None \
        and message.document.file_id is not None and (ext == 'png' or ext == 'jpg'):
            if DEBUG:
                logger(f"User: {chat_id}\tОтправил изображение")
            users[chat_id]['img'] = message.document.file_id
            users[chat_id]['type'] = 'document'
            bot.send_message(chat_id, 'Нажмите кнопку или отправьте новое изображение', reply_markup=gen_markup())
        else:
            if DEBUG:
                logger(f"User: {chat_id}\tОтправил изображение в не правильном формате")
            bot.send_message(chat_id, 'Не верный формат. Нужно отправить изображение в формате jpg и png')
    else:
        bot.send_message(chat_id, 'Сначала вы должны стартовать бот коммандой /start')


# Обработчик для callback'а от inline клавиши
@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    if call is None and call.data is None:
        logger(f"Error in callback")
    elif call.data == "send_back":
        if users[call.from_user.id]['type'] == 'photo':
            bot.send_photo(call.from_user.id, users[call.from_user.id]['img'])
        elif users[call.from_user.id]['type'] == 'document':
            bot.send_document(call.from_user.id, users[call.from_user.id]['img'])

__Задание 3 (1 балл):__ Заменить кнопку возвращения изображения на две кнопки "Размытие" и "Оттенки серого", каждая из которых применяет соответствующее преобразование к сохраненному изображению и возвращает его пользователю. Загрузка нового изображения повторяет логику из предыдущего пункта, кнопки можно нажимать неограниченное количество раз

__Рекомендации:__

1. Указанные фильтры называются "gaussian smoothing" и "grayscale"

__Бонусное задание (до 5 баллов):__ Предложить и реализовать функциональные, эстетические или технические улучшения реализованного бота (допускается только в случае решения всех предыдущих пунктов задания)